# Application Packages in the xcube ecosystem
### EO Summer School 2025

Welcome to the Summer School 2025 course on xcube and Application Packages! This notebook will guide you through the hands-on exercises in the course.

## Requirements

Before you start, look at the requirements list at <https://xcube-dev.github.io/summerschool25/> and make sure that you have the necessary software installed. Open a terminal emulator (either within this JupyterLab or on your desktop) and navigate to the directory containing this notebook.

## 1. Writing a simple application package

### Introducing the CWL file

In this part, we will look at, run, and modify a CWL file which defines a very simple Application Package. It takes no input parameters, and produces as output a single text file containing the text "Hello world!".

Open the file [helloworld.cwl](hello-ap/helloworld.cwl) in this Jupyter Lab or in your favourite text editor. Make sure that you understand the parts: the `Workflow` and `CommandLineTool` sections, the files written into the container, and the command that gets executed in the container.

### Running CWL with `cwltool`

Now let's run it in the terminal. Make sure you have activated the `summerschool25` environment that you created earlier (`conda activate summerschool25`), and that your current directory is the repository directory containing this notebook.

```bash
cwltool --outdir=hello1 hello-ap/helloworld.cwl#hello
```

The outputs will appear in a subdirectory of the `hello1` directory that you specified as `--outdir`. Inside this directory you'll find:

* `myscript.sh`, the one-line program that produced the output.
* `hello.txt`, the output produced by the script.
* `item.json`, a STAC item describing the output file.
* `catalog.json`, a STAC catalogue listing the STAC item.

`cwltool` doesn't care about the STAC data, but a real Application Package platform will use the STAC catalogue to find the output file and stage it out to the user or system that ran the Application Package.

### Try some modifications and additions to the CWL

You can try changing things in the CWL file (e.g. make the shell script write something else) to see the effects when running it. If you want to try a more interesting exercise, add an input parameter – for example, a name that can be inserted in the output text. The parameter has to be defined in four places:

1: In the workflow, replace `inputs: {}` with
```yaml
    inputs:
      name:
        type: string
        default: world
```

2: In the `run_script` subsection of the workflow, replace `in: {}` with
```yaml
        in:
          name: name
```
This maps the workflow parameter `name` to the `CommandLineTool` parameter which is *also* called `name`.

3: In `CommandLineTool`, replace `inputs: {}` with
```yaml
      name:
        type: string
        inputBinding:
          position: 1
```
This tells the `CommandLineTool` to provide the `name` parameter as the first positional argument to the script.

4: In the inline listing for `myscript.sh`, replace `"Hello world!"` with `"Hello $1!"`. This will insert the first positional argument to the script after `Hello`.

## 2. Turning a Jupyter notebook into an Application Package with `xcetool`

### Introducing the input notebook

In the subdirectory `xcengine-nb` of this repository, you will find:
- A notebook called [dynamic.ipynb](xcengine-nb/dynamic.ipynb) which generates some artificial EO data and displays it in the notebook.
- An `environment.yml` file defining the Python environment for the notebook.

First, open the notebook here in Jupyter Lab, look through it, and try running it. The `summerschool25` Python environment already contains all the packages it needs, so you don't need to worry about setting up a special environment for the notebook.

### Turning the notebook into an Application Package

Next, we'll convert the notebook into an Application Package using the `xcetool` command-line utility from the `xcengine` package.

> ℹ️ Normally, you'd push the Docker image for an Application Package to a public registry like Docker Hub, so that it's available for a platform to run. To keep things simple and avoid a potentially slow upload process, we'll keep the image local in this course.

### Adding another parameter